## التضمينات

في المثال السابق، قمنا بالعمل على متجهات حقيبة الكلمات عالية الأبعاد بطول `vocab_size`، وكنا نقوم بتحويل المتجهات ذات التمثيل الموضعي منخفض الأبعاد إلى تمثيل متفرق باستخدام الترميز الواحد. هذا التمثيل الواحد ليس فعالاً من حيث الذاكرة، بالإضافة إلى ذلك، يتم التعامل مع كل كلمة بشكل مستقل عن الأخرى، أي أن المتجهات المشفرة بطريقة الواحد لا تعبر عن أي تشابه دلالي بين الكلمات.

في هذه الوحدة، سنواصل استكشاف مجموعة بيانات **News AG**. للبدء، دعونا نقوم بتحميل البيانات ونسترجع بعض التعريفات من الدفتر السابق.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## ما هو التضمين؟

فكرة **التضمين** هي تمثيل الكلمات بواسطة متجهات كثيفة ذات أبعاد أقل، تعكس بطريقة ما المعنى الدلالي للكلمة. سنتحدث لاحقًا عن كيفية بناء تضمينات كلمات ذات معنى، ولكن في الوقت الحالي دعونا نفكر في التضمينات كطريقة لتقليل أبعاد متجه الكلمة.

لذلك، طبقة التضمين ستأخذ كلمة كمدخل، وتنتج متجهًا كخرج بحجم `embedding_size` المحدد. بمعنى ما، هي مشابهة جدًا لطبقة `Linear`، ولكن بدلاً من أخذ متجه مشفر بطريقة one-hot، ستكون قادرة على أخذ رقم الكلمة كمدخل.

باستخدام طبقة التضمين كأول طبقة في شبكتنا، يمكننا الانتقال من نموذج **bag-of-words** إلى نموذج **embedding bag**، حيث نقوم أولاً بتحويل كل كلمة في النص إلى التضمين المقابل لها، ثم نحسب وظيفة تجميعية معينة لجميع تلك التضمينات، مثل `sum` أو `average` أو `max`.

![صورة توضح مصنف تضمين لخمس كلمات متسلسلة.](../../../../../translated_images/ar/embedding-classifier-example.b77f021a7ee67eee.webp)

شبكة المصنف العصبية الخاصة بنا ستبدأ بطبقة التضمين، ثم طبقة التجميع، ومصنف خطي فوقها:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### التعامل مع حجم تسلسل المتغيرات

نتيجةً لهذه البنية، يجب إنشاء المجموعات الصغيرة (minibatches) لشبكتنا بطريقة معينة. في الوحدة السابقة، عند استخدام تمثيل الحقيبة من الكلمات (bag-of-words)، كانت جميع مصفوفات BoW في المجموعة الصغيرة لها نفس الحجم `vocab_size`، بغض النظر عن الطول الفعلي لتسلسل النص. بمجرد الانتقال إلى تمثيلات الكلمات (word embeddings)، سنجد أن هناك عددًا متغيرًا من الكلمات في كل عينة نصية، وعند دمج هذه العينات في مجموعات صغيرة، سيكون علينا تطبيق بعض الحشو.

يمكن القيام بذلك باستخدام نفس التقنية التي تعتمد على توفير دالة `collate_fn` لمصدر البيانات:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### تدريب مصنف التضمين

الآن بعد أن قمنا بتحديد محمل البيانات بشكل صحيح، يمكننا تدريب النموذج باستخدام وظيفة التدريب التي قمنا بتعريفها في الوحدة السابقة:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **ملاحظة**: نحن نقوم بالتدريب فقط على 25 ألف سجل هنا (أقل من دورة كاملة) من أجل توفير الوقت، ولكن يمكنك متابعة التدريب، كتابة دالة للتدريب لعدة دورات، وتجربة معلمة معدل التعلم لتحقيق دقة أعلى. يجب أن تكون قادرًا على الوصول إلى دقة حوالي 90%.


### طبقة EmbeddingBag وتمثيل التسلسل ذو الطول المتغير

في البنية السابقة، كنا بحاجة إلى ملء جميع التسلسلات لتكون بنفس الطول لكي تتناسب مع دفعة صغيرة. هذه ليست الطريقة الأكثر كفاءة لتمثيل التسلسلات ذات الطول المتغير - هناك نهج آخر يتمثل في استخدام **متجه الإزاحة**، الذي يحتفظ بإزاحات جميع التسلسلات المخزنة في متجه كبير واحد.

![صورة توضح تمثيل تسلسل باستخدام الإزاحة](../../../../../translated_images/ar/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **ملاحظة**: في الصورة أعلاه، نعرض تسلسلًا من الأحرف، ولكن في مثالنا نحن نعمل مع تسلسلات من الكلمات. ومع ذلك، يبقى المبدأ العام لتمثيل التسلسلات باستخدام متجه الإزاحة كما هو.

للعمل مع تمثيل الإزاحة، نستخدم الطبقة [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). إنها مشابهة لـ `Embedding`، لكنها تأخذ متجه المحتوى ومتجه الإزاحة كمدخلات، كما أنها تتضمن طبقة تجميع يمكن أن تكون `mean` أو `sum` أو `max`.

إليك شبكة معدلة تستخدم `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

لتحضير مجموعة البيانات للتدريب، نحتاج إلى توفير وظيفة تحويل ستقوم بتحضير متجه الإزاحة:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

لاحظ أنه على عكس جميع الأمثلة السابقة، تقبل شبكتنا الآن معلمتين: متجه البيانات ومتجه الإزاحة، وهما بحجمين مختلفين. وبالمثل، يوفر لنا محمل البيانات الخاص بنا أيضًا 3 قيم بدلاً من 2: يتم توفير كل من متجهات النص والإزاحة كميزات. لذلك، نحتاج إلى تعديل وظيفة التدريب الخاصة بنا قليلاً للتعامل مع ذلك:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## التضمينات الدلالية: Word2Vec

في المثال السابق، تعلمت طبقة التضمين في النموذج تحويل الكلمات إلى تمثيل متجهي، ولكن هذا التمثيل لم يكن يحمل الكثير من المعاني الدلالية. سيكون من الجيد تعلم تمثيل متجهي بحيث تكون الكلمات المتشابهة أو المرادفات قريبة من بعضها البعض من حيث مسافة معينة بين المتجهات (مثل المسافة الإقليدية).

للقيام بذلك، نحتاج إلى تدريب نموذج التضمين مسبقًا على مجموعة كبيرة من النصوص بطريقة محددة. واحدة من أولى الطرق لتدريب التضمينات الدلالية تُعرف بـ [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). تعتمد هذه الطريقة على اثنين من الهياكل الأساسية التي تُستخدم لإنتاج تمثيل موزع للكلمات:

- **الحقيبة المستمرة للكلمات** (CBoW) — في هذا الهيكل، نقوم بتدريب النموذج لتوقع كلمة بناءً على السياق المحيط. بالنظر إلى النغرام $(W_{-2},W_{-1},W_0,W_1,W_2)$، هدف النموذج هو توقع $W_0$ بناءً على $(W_{-2},W_{-1},W_1,W_2)$.
- **التخطي المستمر للكلمات** (Continuous skip-gram) هو عكس CBoW. يستخدم النموذج نافذة الكلمات المحيطة بالسياق لتوقع الكلمة الحالية.

CBoW أسرع، بينما skip-gram أبطأ ولكنه يقوم بتمثيل الكلمات النادرة بشكل أفضل.

![صورة توضح كلا من خوارزميات CBoW و Skip-Gram لتحويل الكلمات إلى متجهات.](../../../../../translated_images/ar/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

لتجربة تضمين word2vec المدرب مسبقًا على مجموعة بيانات أخبار Google، يمكننا استخدام مكتبة **gensim**. أدناه نجد الكلمات الأكثر تشابهًا مع 'neural'

> **ملاحظة:** عند إنشاء متجهات الكلمات لأول مرة، قد يستغرق تنزيلها بعض الوقت!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


يمكننا أيضًا حساب تضمينات المتجه من الكلمة، لاستخدامها في تدريب نموذج التصنيف (نظهر فقط أول 20 مكونًا من المتجه للتوضيح):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

الشيء العظيم بشأن التضمينات الدلالية هو أنه يمكنك التلاعب بترميز المتجه لتغيير الدلالات. على سبيل المثال، يمكننا أن نطلب العثور على كلمة، يكون تمثيلها المتجهي قريبًا قدر الإمكان من الكلمات *ملك* و *امرأة*، وبعيدًا قدر الإمكان عن الكلمة *رجل*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

كلا من CBoW و Skip-Grams هما تمثيلات "تنبؤية"، حيث يأخذان فقط السياقات المحلية بعين الاعتبار. لا يستفيد Word2Vec من السياق العام.

**FastText** يعتمد على Word2Vec من خلال تعلم تمثيلات متجهية لكل كلمة بالإضافة إلى n-grams للأحرف الموجودة داخل كل كلمة. يتم بعد ذلك حساب متوسط قيم التمثيلات في متجه واحد في كل خطوة تدريب. على الرغم من أن هذا يضيف الكثير من العمليات الحسابية الإضافية أثناء التدريب المسبق، إلا أنه يمكّن تمثيلات الكلمات من تشفير معلومات داخل الكلمة.

طريقة أخرى، **GloVe**، تعتمد على فكرة مصفوفة التكرار المشترك، حيث تستخدم طرقًا عصبية لتحليل مصفوفة التكرار المشترك إلى متجهات كلمات أكثر تعبيرًا وغير خطية.

يمكنك تجربة المثال عن طريق تغيير التمثيلات إلى FastText و GloVe، حيث يدعم gensim عدة نماذج مختلفة لتمثيلات الكلمات.


## استخدام التضمينات المدربة مسبقًا في PyTorch

يمكننا تعديل المثال أعلاه لملء المصفوفة في طبقة التضمين الخاصة بنا مسبقًا بتضمينات دلالية، مثل Word2Vec. يجب أن نأخذ في الاعتبار أن مفردات التضمينات المدربة مسبقًا ومفردات نصوصنا قد لا تتطابق، لذلك سنقوم بتهيئة الأوزان للكلمات المفقودة بقيم عشوائية:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


الآن دعونا ندرب نموذجنا. لاحظ أن الوقت الذي يستغرقه تدريب النموذج أكبر بكثير مقارنة بالمثال السابق، بسبب الحجم الأكبر لطبقة التضمين، وبالتالي عدد أكبر بكثير من المعاملات. أيضًا، بسبب ذلك، قد نحتاج إلى تدريب نموذجنا على المزيد من الأمثلة إذا أردنا تجنب الإفراط في التخصيص.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

في حالتنا، لا نلاحظ زيادة كبيرة في الدقة، وهو ما يرجع على الأرجح إلى اختلاف كبير في المفردات.  
للتغلب على مشكلة اختلاف المفردات، يمكننا استخدام أحد الحلول التالية:  
* إعادة تدريب نموذج word2vec على مفرداتنا  
* تحميل مجموعة البيانات الخاصة بنا باستخدام المفردات من نموذج word2vec المدرب مسبقًا. يمكن تحديد المفردات المستخدمة لتحميل مجموعة البيانات أثناء عملية التحميل.  

النهج الأخير يبدو أسهل، خاصة لأن إطار العمل `torchtext` في PyTorch يحتوي على دعم مدمج للتضمينات. يمكننا، على سبيل المثال، إنشاء مفردات تعتمد على GloVe بالطريقة التالية:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


تم تحميل المفردات مع العمليات الأساسية التالية:  
* قاموس `vocab.stoi` يسمح لنا بتحويل الكلمة إلى رقمها في القاموس  
* يقوم `vocab.itos` بالعكس - يحول الرقم إلى كلمة  
* `vocab.vectors` هو مصفوفة متجهات التضمين، لذا للحصول على تضمين كلمة `s` نحتاج إلى استخدام `vocab.vectors[vocab.stoi[s]]`  

إليك مثال على التلاعب بالتضمينات لإثبات المعادلة **kind-man+woman = queen** (اضطررت إلى تعديل المعامل قليلاً لجعلها تعمل):  


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

لتدريب المصنف باستخدام تلك التضمينات، نحتاج أولاً إلى ترميز مجموعة البيانات الخاصة بنا باستخدام مفردات GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

كما رأينا أعلاه، يتم تخزين جميع تضمينات المتجهات في مصفوفة `vocab.vectors`. يجعل ذلك من السهل جدًا تحميل هذه الأوزان في أوزان طبقة التضمين باستخدام النسخ البسيط:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

الآن دعونا ندرب نموذجنا ونرى إذا حصلنا على نتائج أفضل:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

أحد الأسباب التي تجعلنا لا نرى زيادة كبيرة في الدقة هو حقيقة أن بعض الكلمات من مجموعة البيانات الخاصة بنا مفقودة في مفردات GloVe المدربة مسبقًا، وبالتالي يتم تجاهلها بشكل أساسي. للتغلب على هذه المشكلة، يمكننا تدريب التضمينات الخاصة بنا على مجموعة البيانات الخاصة بنا.


## التضمينات السياقية

أحد القيود الرئيسية لتمثيلات التضمينات المدربة مسبقًا مثل Word2Vec هو مشكلة تمييز معاني الكلمات. على الرغم من أن التضمينات المدربة مسبقًا يمكنها التقاط بعض معاني الكلمات في السياق، إلا أن كل المعاني الممكنة للكلمة يتم ترميزها في نفس التضمين. يمكن أن يسبب هذا مشاكل في النماذج اللاحقة، حيث أن العديد من الكلمات مثل كلمة "play" لها معانٍ مختلفة تعتمد على السياق الذي تُستخدم فيه.

على سبيل المثال، كلمة "play" في الجملتين التاليتين لها معانٍ مختلفة تمامًا:
- ذهبت إلى **عرض مسرحي** في المسرح.
- جون يريد أن **يلعب** مع أصدقائه.

التضمينات المدربة مسبقًا أعلاه تمثل كلا المعنيين لكلمة "play" في نفس التضمين. للتغلب على هذا القيد، نحتاج إلى بناء تضمينات تعتمد على **نموذج اللغة**، الذي يتم تدريبه على مجموعة كبيرة من النصوص، و*يعرف* كيف يمكن للكلمات أن تتجمع في سياقات مختلفة. مناقشة التضمينات السياقية خارج نطاق هذا الدرس، لكننا سنعود إليها عند الحديث عن نماذج اللغة في الوحدة التالية.



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
